In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import os

from synthesizer.grid import Grid
from synthesizer.parametric import SFH, ZDist
from synthesizer.particle.stars import sample_sfhz
from synthesizer.parametric import Stars as ParametricStars
from synthesizer.particle.particles import CoordinateGenerator
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds

from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

from synthesizer.conversions import lnu_to_absolute_mag

/tmp/ipykernel_520/84371063.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# get grid for all galaxies
# gives grid of metalicity and age which is used to map on our camels galaxies
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
#grid_dir = "/Users/am/Documents/CAMELS/data/grids"
grid_dir = "/disk/xray15/aem2/grids"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

In [4]:
# To get apparent Magnitudes
# check 
from astropy.cosmology import Planck18    # You can choose other cosmologies if needed

def apparent_magnitude_from_absolute(redshift, absolute_magnitude):
    # Get the luminosity distance for the given redshift
    # Distnaces are in parsecs
    d_L_pc = Planck18.luminosity_distance(redshift).to('pc').value
    
    # Calculate the apparent magnitude using the distance modulus formula
    apparent_mag = absolute_magnitude - 5 + (5 * np.log10(d_L_pc))
    
    return apparent_mag

In [15]:
def make_fld(app_mag, label):

    header_lines = [
        "  0.0    0.0    23.91      5.0",
        "  0.0119999",
        "          3     700   3      0.0 0.0 0.0",
        "  fid   sid   dummyXflux   z        mag",
    ]
    # TOP HAT
    # Format the data lines
    data_lines = ["  0     0     0            0.46525   {:.17f}".format(val) for val in app_mag]

    # Combine header lines and data lines
    all_lines = "\n".join(header_lines) + "\n" + "\n".join(data_lines)

    file_name = label + '.fld'
    # Save to file
    with open(file_name, 'w') as file:
        file.write(all_lines)

# NOTE: only using TOPHAT filter for now

In [6]:
def load_data(par_dir):
    # get gals - all 074 (redshift 0.475)
    gals = load_CAMELS_IllustrisTNG(
        #"/Users/am/Documents/CAMELS/data/1P/1P_1p_0",
        par_dir,
        snap_name="snapshot_074.hdf5", # name change on server (snapshot_025)
        fof_name="groups_074.hdf5", # name change on server (groups_025)
    )

    print('Number of Galaxies', len(gals))

    cat = os.path.join(par_dir, 'groups_074.hdf5')
    # open file
    f = h5py.File(cat, 'r')

    # read different attributes of the header
    boxSize = f['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
    redshift = f['Header'].attrs[u'Redshift']
    print(redshift)
    # return boxSize, redshift, gals
    # NOTE: 074 is the same redshift as CV_0/025
    
    # working up to here
    # Filter galaxies to only include those with 100 or more star particles
    ## NEED TO ADD FILTER TO GET RID OF ANY GALAXIES WITH LESS THAN 100 star Particles!! This is the resolution limit!!
    # Chris: when you load your galaxies in could you do a filter for those with at least 100 star particles? that's the resolution limit
    gals = [gal for gal in gals if len(gal.stars.ages) >= 100]
    print('Number of Galaxies aafter resolution limit applied', len(gals))

    spec_list = []
    # Lets work with z=0.475 so gals_p10
    for i in gals:
        gal = i
        # get_spectra_incident An Sed object containing the stellar spectra
        spec = gal.stars.get_spectra_incident(grid)
        spec.get_fnu0()
        spec_list.append(spec)
        # combine
    seds = combine_list_of_seds(spec_list)
    
    # Need to use the grid lam, dont need to define wavelengths here: see codes here: http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Swift/UVOT.UVM2&&mode=browse&gname=Swift&gname2=UVOT#filter
    # need to use transmission NOT area
    # Get for XMM
    #fil_uvm2_XMM = Filter("XMM/OM.UVM2_filter", new_lam=grid.lam)

    # now get for UVOT
    #fil_uvm2_UVOT = Filter("Swift/UVOT.UVM2_fil", new_lam=grid.lam) # changed from new_lam=lams_uvot to grid.lam and get 0 transmission, but warning sign is back

    # what is a top hat filter?
    filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)

    # 0 = TopHat, 1 = XMM, 2= Swift
    filt_lst = [filt1]#, fil_uvm2_XMM, fil_uvm2_UVOT]

    # get filters in combined array to apply to all galaxies
    combined_filters = FilterCollection(
    filters=filt_lst, new_lam=grid.lam
    )
    
    seds.lnu # rest frame lumd
    
    seds.get_photo_luminosities(combined_filters)

    seds.photo_luminosities.photo_luminosities

    abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)

    app_mag = apparent_magnitude_from_absolute(redshift, abs_mag)

    app_mag_th = app_mag[0] # 0 is tophat
    #app_mag_xmm = app_mag[1] # 1 is XMM-OM
    #app_mag_uvot = app_mag[2] # 2 is Swift UVOT

    return gals, redshift, app_mag_th#, app_mag_xmm, app_mag_uvot - dont need these for now

# 1P_p1_X = $\Omega_{m}$

In [7]:
gals_1P_p1_0, redshift_1P_p1_0, app_mag_1P_p1_0 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p1_0")
gals_1P_p1_1, redshift_1P_p1_1, app_mag_1P_p1_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p1_1")
gals_1P_p1_n1, redshift_1P_p1_n1, app_mag_1P_p1_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p1_n1")
gals_1P_p1_n2, redshift_1P_p1_n2, app_mag_1P_p1_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p1_n2")

make_fld(app_mag_1P_p1_0, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p1/outfiles/app_mag_1P_p1_0')
make_fld(app_mag_1P_p1_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p1/outfiles/app_mag_1P_p1_1')
make_fld(app_mag_1P_p1_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p1/outfiles/app_mag_1P_p1_n1')
make_fld(app_mag_1P_p1_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p1/outfiles/app_mag_1P_p1_n2')

/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1938: RuntimeWarning: invalid value encountered in divide
  out_arr = func(


Number of Galaxies 20225
0.46525063335034633
Number of Galaxies aafter resolution limit applied 420
Number of Galaxies 21264
0.46525063335034633
Number of Galaxies aafter resolution limit applied 481


/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: invalid value encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)


Number of Galaxies 17717
0.46525063335034633
Number of Galaxies aafter resolution limit applied 278


/srv/conda/envs/notebook/lib/python3.9/site-packages/unyt/array.py:1813: RuntimeWarning: divide by zero encountered in log10
  out_arr = func(np.asarray(inp), out=out_func, **kwargs)


Number of Galaxies 11329
0.4643830386955963
Number of Galaxies aafter resolution limit applied 110


# 1P_p2_Y = $\sigma_{8}$

In [ ]:
gals_1P_p2_1, redshift_1P_p2_1, app_mag_1P_p2_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p2_1")
gals_1P_p2_2, redshift_1P_p2_2, app_mag_1P_p2_2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p2_2")
gals_1P_p2_n1, redshift_1P_p2_n1, app_mag_1P_p2_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p2_n1")
gals_1P_p2_n2, redshift_1P_p2_n2, app_mag_1P_p2_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p2_n2")

make_fld(app_mag_1P_p2_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p2/outfiles/app_mag_1P_p2_1')
make_fld(app_mag_1P_p2_2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p2/outfiles/app_mag_1P_p2_2')
make_fld(app_mag_1P_p2_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p2/outfiles/app_mag_1P_p2_n1')
make_fld(app_mag_1P_p2_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p2/outfiles/app_mag_1P_p2_n2')

# 1P_p3_Y = $A_{SN1}$

In [9]:
gals_1P_p3_1, redshift_1P_p3_1, app_mag_1P_p3_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p3_1")
gals_1P_p3_2, redshift_1P_p3_2, app_mag_1P_p3_2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p3_2")
gals_1P_p3_n1, redshift_1P_p3_n1, app_mag_1P_p3_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p3_n1")
gals_1P_p3_n2, redshift_1P_p3_n2, app_mag_1P_p3_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p3_n2")

make_fld(app_mag_1P_p3_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p3/outfiles/app_mag_1P_p3_1')
make_fld(app_mag_1P_p3_2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p3/outfiles/app_mag_1P_p3_2')
make_fld(app_mag_1P_p3_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p3/outfiles/app_mag_1P_p3_n1')
make_fld(app_mag_1P_p3_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p3/outfiles/app_mag_1P_p3_n2')

Number of Galaxies 20012
0.46525063335034633
Number of Galaxies aafter resolution limit applied 223
Number of Galaxies 20296
0.46525063335034633
Number of Galaxies aafter resolution limit applied 115
Number of Galaxies 19926
0.46525063335034633
Number of Galaxies aafter resolution limit applied 627
Number of Galaxies 20031
0.46525063335034633
Number of Galaxies aafter resolution limit applied 919


In [12]:
app_mag_1P_p3_n1

array([21.1021428 , 26.70702281, 27.76670526, 28.14985897, 28.74668272,
       28.07191333, 27.86355393, 28.14551298, 23.55050444, 28.18671998,
       21.22640495, 28.56269396, 28.64109051, 29.00943945, 27.28011683,
       28.53370392, 24.10515297, 29.41043995, 23.93508208, 24.40922956,
       29.27815597, 29.83033078, 29.32214864, 29.63073048, 29.29751879,
       23.99874167, 29.56741718, 28.39801865, 30.30249467, 29.69528198,
       21.89656577, 29.9587989 , 30.09426035, 30.49578895, 30.56936344,
       31.5485233 , 30.71386154, 30.80728793, 30.9322249 , 30.58865648,
       31.30747396, 31.32700352, 21.93771668, 31.30727895, 30.83052557,
       31.06784164, 28.44194728, 30.78433483, 30.67855451, 31.96831663,
       31.8865211 , 31.3102647 , 31.2740441 , 31.4204841 , 31.74761596,
       31.94293318, 32.08476268, 31.51560629, 32.06182029, 32.49795808,
       31.98036187, 27.14093223, 27.74260394, 27.02535126, 24.12731283,
       22.05117638, 26.15101033, 25.20566409, 31.07381432, 28.23

In [11]:
app_mag_1P_p3_n2

array([22.95120502, 26.35395711, 26.58718843, 27.75990641, 27.73257902,
       27.67888648, 22.41592918, 28.89713209, 27.88900257, 27.55436786,
       28.02025081, 27.96604504, 28.16232775, 21.1324984 , 28.41337527,
       29.17185065, 28.90321339, 28.71293695, 26.10409491, 22.89025715,
       29.10050811, 22.57665969, 28.60108823, 27.86753908, 29.02919081,
       29.08071371, 22.17637086, 24.47999038, 29.38033679, 29.0984233 ,
       29.77726594, 23.25869601, 29.45483514, 29.55337441, 29.15693846,
       29.70791271, 22.01489801, 27.25161457, 30.15812551, 30.29047961,
       29.83083622, 30.18413444, 30.56476551, 30.41885938, 27.51431874,
       24.88749949, 30.52272224, 30.58210977, 30.4590725 , 30.32716541,
       27.18850631, 29.95372894, 22.49161207, 30.3913292 , 31.05485612,
       30.55201318, 30.49173588, 27.56702002, 30.53090953, 31.86965752,
       31.88594373, 31.09349735, 31.28330599, 30.84643959, 31.57309644,
       31.2926189 , 30.83068592, 31.09647751, 31.39307991, 31.32

# 1P_p5_Y = $A_{SN2}$

In [17]:
gals_1P_p5_1, redshift_1P_p5_1, app_mag_1P_p5_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p5_1")
gals_1P_p5_2, redshift_1P_p5_2, app_mag_1P_p5_2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p5_2")
gals_1P_p5_n1, redshift_1P_p5_n1, app_mag_1P_p5_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p5_n1")
gals_1P_p5_n2, redshift_1P_p5_n2, app_mag_1P_p5_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p5_n2")

make_fld(app_mag_1P_p5_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p5/outfiles/app_mag_1P_p5_1')
make_fld(app_mag_1P_p5_2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p5/outfiles/app_mag_1P_p5_2')
make_fld(app_mag_1P_p5_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p5/outfiles/app_mag_1P_p5_n1')
make_fld(app_mag_1P_p5_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p5/outfiles/app_mag_1P_p5_n2')

Number of Galaxies 20112
0.46525063335034633
Number of Galaxies aafter resolution limit applied 372
Number of Galaxies 20576
0.46525063335034633
Number of Galaxies aafter resolution limit applied 392
Number of Galaxies 19821
0.46525063335034633
Number of Galaxies aafter resolution limit applied 415
Number of Galaxies 19946
0.46525063335034633
Number of Galaxies aafter resolution limit applied 421


# 1P_p4_Y = $A_{AGN1}$

In [18]:
gals_1P_p4_1, redshift_1P_p4_1, app_mag_1P_p4_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p4_1")
gals_1P_p4_2, redshift_1P_p4_2, app_mag_1P_p4_2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p4_2")
gals_1P_p4_n1, redshift_1P_p4_n1, app_mag_1P_p4_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p4_n1")
gals_1P_p4_n2, redshift_1P_p4_n2, app_mag_1P_p4_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p4_n2")

make_fld(app_mag_1P_p4_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p4/outfiles/app_mag_1P_p4_1')
make_fld(app_mag_1P_p4_2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p4/outfiles/app_mag_1P_p4_2')
make_fld(app_mag_1P_p4_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p4/outfiles/app_mag_1P_p4_n1')
make_fld(app_mag_1P_p4_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p4/outfiles/app_mag_1P_p4_n2')

Number of Galaxies 20028
0.46525063335034633
Number of Galaxies aafter resolution limit applied 413
Number of Galaxies 20220
0.46525063335034633
Number of Galaxies aafter resolution limit applied 423
Number of Galaxies 19936
0.46525063335034633
Number of Galaxies aafter resolution limit applied 393
Number of Galaxies 20188
0.46525063335034633
Number of Galaxies aafter resolution limit applied 406


# 1P_p6_Y = $A_{AGN2}$

In [19]:
gals_1P_p6_1, redshift_1P_p6_1, app_mag_1P_p6_1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p6_1")
gals_1P_p6_2, redshift_1P_p6_2, app_mag_1P_p6_2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p6_2")
gals_1P_p6_n1, redshift_1P_p6_n1, app_mag_1P_p6_n1 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p6_n1")
gals_1P_p6_n2, redshift_1P_p6_n2, app_mag_1P_p6_n2 = load_data("/home/jovyan/Data/Sims/IllustrisTNG/1P/1P_p6_n2")

make_fld(app_mag_1P_p6_1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p6/outfiles/app_mag_1P_p6_1')
make_fld(app_mag_1P_p6_2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p6/outfiles/app_mag_1P_p6_2')
make_fld(app_mag_1P_p6_n1, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p6/outfiles/app_mag_1P_p6_n1')
make_fld(app_mag_1P_p6_n2, '/home/jovyan/camels/play/synth-play/1P_parameters/1P_p6/outfiles/app_mag_1P_p6_n2')

Number of Galaxies 20018
0.46525063335034633
Number of Galaxies aafter resolution limit applied 412
Number of Galaxies 20196
0.46525063335034633
Number of Galaxies aafter resolution limit applied 423
Number of Galaxies 20006
0.46525063335034633
Number of Galaxies aafter resolution limit applied 394
Number of Galaxies 20203
0.46525063335034633
Number of Galaxies aafter resolution limit applied 434
